# PART I: COLLECTING DATA


1. Loading all necessary tools and libraries:



In [ ]:

from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import io 
import glob 
import numpy as np
import pandas as pd
import math
import random
import sys

At first step males and females rankings were dowloaded usind Pandas and read_html function and saved to .csv files

In [ ]:
## create dataFrame by readeing table from link:
male_df = pd.read_html('https://www.8a.nu/ranking/sportclimbing?gender=male')[0]
for i in range(2,149):
   ## add to dataFrame tables from each of rankings pages
    ex_tmp = pd.read_html('https://www.8a.nu/ranking/sportclimbing?topAscents=top10&gender=male&style=combined&country=&time=lastyear&age=combined&page=' + str(i), encoding='utf-8')[0]
    male_df=pd.concat([male_df,ex_tmp])

## add column with specific name format -- required in next steps
male_df["CLIMBER_NICK"] = male_df["CLIMBER"].str.replace(" ","-").str.lower()
## save to csv
male_df.to_csv('male2_8a_df_all.csv')

## The same for females
female_df = pd.read_html('https://www.8a.nu/ranking/sportclimbing?gender=female')[0]
for i in range(2,32):
    female_tmp = pd.read_html('https://www.8a.nu/ranking/sportclimbing?gender=female&page=' + str(i))[0]
    female_df=pd.concat([female_df,female_tmp])

female_df["CLIMBER_NICK"] = female_df["CLIMBER"].str.replace(" ","-").str.lower()
female_df.to_csv('female_8a_df_all.csv')

Selenium IDE was used to get specific information about each of climber, because there was no posibility to use selenium in python script.



Informations about one user where dowloaded as single .csv file and saved in folder Male and Female. Based on single .csv file specific informations where calculated:


1.   Boulder / Sport - check if climber is sport climber or boulderer (compare amount of boulder routes and sport routes made by user)
2.   Region - Which climbing region is the most common for that person
3.   Grade - There is specific climbing routes scale which had to be transformed into numeric scale, and then mean grade for that user was calculated
4.   Years - based on dates of climbed routes where calculated how long each user has been climbing
5.   RoutesNbr - How many routes user made
6.   OS/ FLASH/ RP - are the styles of climbing - depends on them user get apropriate number of points for that route
7.   <6a - =>8a - scale range of routes
8.   MAX - maximum grade which climber has made

In [ ]:
# create empty dataFrame
dfComparison = pd.DataFrame(columns=['Years', 'MAX', 'RoutesNbr', "OS", "Flash", "RP", "Region","<6a" , "6a-6c+", "7a-7c+", "=>8a", "Grade"])

# get all filenames of males
local_path = r'/content/drive/MyDrive/DM_CLIMBING/males2/'
filenames = glob.glob(local_path + "/*.csv")

local_path_f = r'/content/drive/MyDrive/DM_CLIMBING/females/'
filenames_f = glob.glob(local_path_f + "/*.csv")

In [ ]:
# function to calculate all mentioned informations based on one user .csv file 
def MakeDict(dfInput):
  try: 
    dfSport = dfInput[dfInput.type != "boulder"]
    dfBoulder = dfInput[dfInput.type == "boulder"]
    lenBoulder = len(dfBoulder.index)
    lenSport = len(dfSport.index)
    if lenBoulder > lenSport:
      dfAnalyse = dfBoulder
    else:
      dfAnalyse = dfSport
    commonRegion = dfAnalyse['region'].value_counts().index[0]
    dateGrade = dfSport[['date', 'grade','style']].sort_values(by='date')
    dateGrade['date'] = dateGrade['date'].astype(str).str[0:4]
    dateGrade = dateGrade.replace({'grade': gradeDict})
    less6a = dateGrade.grade[dateGrade.grade < 6.1].count()
    start6a_6cplus = dateGrade.grade[(dateGrade.grade >= 6.1) & (dateGrade.grade <= 6.25)].count()
    start7a_7cplus = dateGrade.grade[(dateGrade.grade >= 7.0) & (dateGrade.grade < 8.0)].count()
    more8a = dateGrade.grade[dateGrade.grade >= 8.0].count()
    dg = dateGrade[['date', 'grade']]
    dfSingleStats = dateGrade
    dfSingleStats['Flash'] = dfSingleStats['style'].apply(lambda x: 1 if x == 'flash' else 0)
    dfSingleStats['OS'] = dfSingleStats['style'].apply(lambda x: 1 if x == 'onsight' else 0)
    dfSingleStats['RP'] = dfSingleStats['style'].apply(lambda x: 1 if x == 'redpoint' else 0) 
    dfSingleStats = dfSingleStats.groupby(by=["date"]).sum()
    dfSingleStats['grade'] = dateGrade.groupby(by=["date"]).mean().round(2)
    dfSingleStats['count'] = dateGrade.groupby(by=["date"]).count()['grade']
    dfSingleStats['Flash'] = (dfSingleStats['Flash']/dfSingleStats['count']).round(3)
    dfSingleStats['OS'] = (dfSingleStats['OS']/dfSingleStats['count']).round(3)
    dfSingleStats['RP'] = (dfSingleStats['RP']/dfSingleStats['count']).round(3)
    dfSingleStats['MAX'] = dg.sort_values(by='date').groupby(by=["date"]).max()
    singleDict = {"Years": len(dfSingleStats.index), "MAX": dfSingleStats['MAX'].max(), "RoutesNbr": dfSingleStats['count'].sum(), "OS": round(dfSingleStats['OS'].mean(),3), "Flash": round(dfSingleStats['Flash'].mean(),3),  "RP": round(dfSingleStats['RP'].mean(),3), "Region":commonRegion, "<6a":less6a , "6a-6c+":start6a_6cplus, "7a-7c+":start7a_7cplus, "=>8a":more8a, "Grade": round(dfSingleStats['grade'].mean(),3)}
  except:
    singleDict = {"Years": float("NaN"), "MAX": float("NaN"), "RoutesNbr": float("NaN"), "OS": float("NaN"), "Flash": float("NaN"),  "RP": float("NaN"), "Region":float("NaN"), "<6a":float("NaN"), "6a-6c+":float("NaN"), "7a-7c+":float("NaN"), "=>8a":float("NaN"), "Grade": float("NaN")}
  return singleDict


In [ ]:
# apply funtion to all files and add calculated informations to dataFrame
incr = 0
for fileNbr in filenames:
  singleDict = {}
  try:
    df = pd.read_csv(fileNbr)
    singleDict = MakeDict(df)
    valuesList = list(singleDict.values())
  except:
    valuesList = np.empty((12))
    valuesList.fill(np.nan)
  dfComparison.loc[incr] = valuesList
  incr += 1

In [ ]:
# females
df = pd.read_csv(filenames_f[0])
for fileNbr_f in filenames_f:
  try:
    df2 = pd.read_csv(fileNbr_f)
    df = pd.concat([df, df2])
  except:
    pass


In [ ]:
# males
for fileNbr_f in filenames:
  try:
    df2 = pd.read_csv(fileNbr_f)
    df = pd.concat([df, df2])
  except:
    pass

In [ ]:
# dictionary to transform climbing grades into numeric scale:
gradeDict = {'3a': 3.0, '3b': 3.0, '3c': 3.0, '4a': 4.0, '4b': 4.0, '4c': 4.0,  "4":4.0, "5a":5.0, "5a+":5.05, "5b":5.1, "5b+":5.15, "5c":5.2, "5c+":5.25, "6a":6.0, "6a+":6.05,"6b":6.1, "6b+":6.15, "6c":6.2,"6c+":6.25, "7a":7.0, "7a+":7.05, "7b":7.1, "7b+":7.15, "7c":7.2, "7c+":7.25, "8a":8.0, "8a+":8.05, "8b": 8.1, "8b+":8.15, "8c":8.2, "8c+":8.25, "9a":9.0, "9a+":9.05, "9b":9.1, "9b+": 9.15, "9c":9.2}

In [ ]:
df["grade"] = df["grade"].str.lower()
dateGrade = df[['grade']]

# translate ordinal scale into numeric based on dictionary above:
dateGrade = dateGrade.replace({'grade': gradeDict})
df = df.drop('grade', axis = 1)
df = pd.concat([df, dateGrade], axis=1)

# save created data frame to csv file:
df.to_csv('/content/drive/MyDrive/DM_CLIMBING/all_joined.csv')